In [ ]:
# Se importa uvicorn para poder ejecutar la aplicación desde el notebook
import uvicorn

# Archivo principal de la aplicación
# 1. Importar FastAPI
# 2. Importamos HTTPException para poder devolver errores HTTP
from fastapi import FastAPI
from fastapi import HTTPException

# Importamos las tareas que queremos ejecutar de manera asíncrona
from tasks import add, mul


In [ ]:
# 3. Crear una instancia de FastAPI
# title: Título de la API
# description: Descripción de la API
# version: Versión de la API
app = FastAPI(title="DiegoDG.com.mx", description="API para probar FastAPI, Redis y Celery", version="1.0")

In [ ]:
# 4. Creamos unas rutas sencilla
# get: Método HTTP
# /: Ruta
@app.get("/")
def index():
    return {"message": "Hola Mundo"}

# get: Método HTTP
# /suma/{a}/{b}: Ruta con parámetros
@app.get("/suma/{a}/{b}")
def suma(a: int, b: int):
    try:
        # La funcion add tiene una función delay que permite enviar la tarea a Celery
        add.delay(a, b)
        # devolvemos un mensaje para saber que se ha enviado la tarea a Celery.
        # Esto no quiere decir que se haya ejecutado correctamente la tarea.
        # Nuestra tarea puede fallar en Celery es por ello que debemos de tener un manejo de errores
        # y almacenar su resultado si es necesario
        return {"message": "Se ha enviado la tarea de sumar a Celery"}
    except Exception as e:
        # Si hay un error durante el envío de la tarea a Celery, devolvemos un error HTTP
        raise HTTPException(status_code=400, detail=str(e))


# get: Método HTTP
# /multiplicacion/{a}/{b}: Ruta con parámetros
@app.get("/multiplicacion/{a}/{b}")
def multiplicacion(a: int, b: int):
    try:
        mul.delay(a, b)
        return {"message": "Se ha enviado la tarea de multiplicar a Celery"}
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

In [ ]:
# Configuracion para poder ejecutar la aplicación desde el notebook
# Si se ejecuta desde un archivo .py, no es necesario este bloque de código
if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    await server.serve()